In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [3]:
##### Load the dataset
data = pd.read_csv('Employee_Exit_Data.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## processthe data
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
## Encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [ ]:
## one hot encode the 'Geography' column
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder_geography = OneHotEncoder()
geo_encoder = one_hot_encoder_geography.fit_transform(data[['Geography']]).toarray()
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [7]:
one_hot_encoder_geography.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [8]:
geo_encoder_df = pd.DataFrame(geo_encoder, columns=one_hot_encoder_geography.get_feature_names_out(['Geography']))
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
## combine the one hot encoder columns with the original data
data = pd.concat([data, geo_encoder_df], axis=1)
data = data.drop(['Geography'], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
## create pickel files for the label encoder and one hot encoder
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('one_hot_encoder_geography.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder_geography, file)

# 'wb' stands for write binary mode.
# It opens the file for writing in binary format.
# This mode is required when saving objects with pickle.dump(), which serializes Python 
# objects into binary data.
# Use 'wb' whenever you need to write binary data (like pickled objects) to a file.

In [11]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [12]:
## divide the dataset into features(dependent) and target(indepandend) variable
X = data.drop(['Exited'], axis=1)
y = data['Exited']

## split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# test_size=0.2: This means 20% of the data will be used for testing, and 80% for training.
# random_state=42: This sets the seed for random number generation, ensuring the split 
# is reproducible. Using the same random_state will always give the same train/test split.

## scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### scaler.fit_transform(X_train) fits the scaler to the training 
# data (calculates mean and std) and then transforms it (scales the data).
#### scaler.transform(X_test) only transforms the test data using the 
# mean and std calculated from the training data.

In [13]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [14]:
X_test

array([[-0.57749609,  0.91324755, -0.6557859 , ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.29729735,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.52560743, -1.09499335,  0.48508334, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.81311987, -1.09499335,  0.77030065, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.41876609,  0.91324755, -0.94100321, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.24540869,  0.91324755,  0.00972116, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [15]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [16]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


ANN Implementation


In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


In [18]:
(X_train.shape[1],)

# (X_train.shape[1],) returns a tuple with one element: the number of features (columns) in your training data.

# X_train.shape[1] gives the count of columns (features) in X_train.
# Wrapping it in parentheses and a comma, (X_train.shape[1],), creates a tuple.
# This format is often used to specify the input shape for neural network layers in Keras/TensorFlow

(12,)

In [19]:
X_train.shape

# X_train.shape returns a tuple: (number_of_rows, number_of_columns)
# [1] accesses the second value, which is the number of features.
# For example, if X_train.shape is (8000, 12), then X_train.shape[1] is 12.
# This is useful for specifying the input size for neural network models.

(8000, 12)

In [20]:
### Build the ANN model
from tensorflow.keras.layers import Input
model = Sequential([
    Input(shape=(X_train.shape[1],)),           # Explicit Input layer
    Dense(64, activation='relu'),               # Hidden Layer 1
    Dense(32, activation='relu'),               # Hidden Layer 2
    Dense(1, activation='sigmoid')              # Output Layer
])

# The Sequential model in Keras is used to build neural networks layer-by-layer in a linear stack.
# You add layers in order, and data flows through each layer sequentially.
# It is simple and best for models where each layer has one input tensor and one output tensor.

# This line defines a dense (fully connected) layer in a neural network using Keras:
# Dense(64, ...) creates a layer with 64 neurons.
# activation='relu' sets the activation function to ReLU (Rectified Linear Unit), 
# which helps the model learn non-linear relationships.
# input_shape=(X_train.shape[1],) tells the layer to expect input data with a number of 
# features equal to the number of columns in X_train. This is required for the first 
# layer in the model to know the shape of the input data

In [21]:
model.summary()

# model.summary() displays the architecture of your neural network, including each layer, 
# its output shape, and the number of parameters.

# Parameters in this context are the weights and biases that the model will learn during 
# training:

# For a Dense layer:
# Parameters = (number of input units × number of output units) + number of output units (biases)
# Example for Dense(64, activation='relu') after an input layer with 12 features:

# Parameters = (12 × 64) + 64 = 832
# Each layer's parameters are summed to give the Total params at the bottom of the summary.

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
optmzr = tf.keras.optimizers.Adam(learning_rate=0.001)
# Adam optimizer is an adaptive learning rate optimization algorithm that is widely used in training deep learning models.
# It combines the advantages of two other extensions of stochastic gradient descent, AdaGrad and RMSProp
loss = tf.keras.losses.BinaryCrossentropy()
# BinaryCrossentropy is a loss function used for binary classification tasks.


In [23]:
## compile the model
model.compile(optimizer=optmzr, loss=loss, metrics=['accuracy'])

# metrics=['accuracy'] tells Keras to report the accuracy of your model during training and 
# evaluation.

# Accuracy measures the proportion of correct predictions out of all predictions.
# It helps you monitor how well your model is performing on the training and validation data.
# You can add other metrics if needed, but 'accuracy' is the most common for classification tasks.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
# EarlyStopping is a callback that stops training when a monitored metric has stopped improving.
# It helps prevent overfitting by stopping the training process when the model's performance
# on the validation set does not improve for a specified number of epochs.

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

# TensorBoard is a visualization tool that helps you monitor and visualize the training process of your model.
# It provides insights into metrics like loss and accuracy, as well as visualizations of the model
# architecture, gradients, and more.

In [28]:
## set the early stopping criteria
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# EarlyStopping monitors the validation loss ('val_loss') during training.
# If the validation loss does not improve for a specified number of epochs (patience=5),
# it stops the training process and restores the best weights from the epoch with the lowest validation loss
# EarlyStopping helps prevent overfitting by stopping training when the model's performance on the validation set stops improving.
# It also restores the best weights to ensure the model is in its best state when training stops.



In [29]:
## Train the model
history = model.fit(X_train, y_train, validation_data=(X_test,y_test),epochs=100, batch_size=32, validation_split=0.2,
                    callbacks=[early_stopping_callback, tensorflow_callback])  

# The model.fit() method trains the model on the training data (X_train, y_train).
# validation_data=(X_test, y_test) specifies the validation set to evaluate the model's performance after each epoch.
# epochs=100 means the model will go through the entire training dataset 100 times. 
# batch_size=32 means the model will update its weights after processing 32 samples at a time.
# validation_split=0.2 means 20% of the training data will be used for validation during training.
# callbacks=[early_stopping_callback, tensorflow_callback] allows you to use callbacks like EarlyStopping and TensorBoard during training.
# # --- IGNORE ---
# # EarlyStopping monitors the validation loss and stops training if it doesn't improve for 5 epochs.
# # # TensorBoard logs the training process for visualization.
# # The history object contains the training and validation loss and accuracy for each epoch, which can be used for further analysis or visualization.
# # # --- IGNORE ---  

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.7974 - loss: 0.4899 - val_accuracy: 0.8300 - val_loss: 0.4006
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8326 - loss: 0.4007 - val_accuracy: 0.8505 - val_loss: 0.3625
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8561 - loss: 0.3559 - val_accuracy: 0.8560 - val_loss: 0.3490
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8649 - loss: 0.3346 - val_accuracy: 0.8570 - val_loss: 0.3441
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8615 - loss: 0.3388 - val_accuracy: 0.8610 - val_loss: 0.3378
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8603 - loss: 0.3369 - val_accuracy: 0.8620 - val_loss: 0.3383
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8633 - loss: 0.3237 - val_accuracy: 0.8595 - val_loss: 0.3409
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8636 - loss: 0.3342 - val_acc

In [30]:
model.save('model.h5')
# model.save('model.h5') saves the trained model to a file named 'model.h5'.
# This file can be loaded later to make predictions without needing to retrain the model.   

In [33]:
## Load Tensorboard Extension
%load_ext tensorboard
%tensorboard --logdir logs/fit
# %load_ext tensorboard loads the TensorBoard extension in Jupyter Notebook.    
# %tensorboard --logdir logs/fit starts the TensorBoard server and points it to the log directory where training logs are stored.
# This allows you to visualize the training process, including metrics like loss and accuracy, as well

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 15460), started 0:00:48 ago. (Use '!kill 15460' to kill it.)